In [82]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import re
from sklearn.feature_extraction.text import CountVectorizer

In [83]:
all_verbs = pd.read_csv('./all_verbs_LR.csv').reset_index(drop = True)
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e
...,...,...,...,...,...,...,...,...,...,...,...,...
44107,44107,past,subjunctive,pluperfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
44108,44108,past,subjunctive,pluperfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
44109,44109,past,subjunctive,pluperfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ
44110,44110,past,subjunctive,pluperfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ


In [84]:
all_verbs['Stress Number'] = all_verbs['Generated Verb-SR'].apply(lambda word: word.rindex('ˈ') / len(word)) #before : goes input variable
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44107,44107,past,subjunctive,pluperfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455
44108,44108,past,subjunctive,pluperfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455
44109,44109,past,subjunctive,pluperfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ,0.500000
44110,44110,past,subjunctive,pluperfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ,0.500000


In [85]:
from scipy.sparse import hstack
count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
X = count_vect.fit_transform(all_verbs['Generated Verb-SR'])


In [86]:
hstack([X, all_verbs['Stress Number'].values.reshape(-1,1)]) 

<44112x1668 sparse matrix of type '<class 'numpy.float64'>'
	with 1614182 stored elements in COOrdinate format>

In [87]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:


        model = LogisticRegression(max_iter=100000) 
        X = all_verbs[['Generated Verb-SR', 'Stress Number']] 
        y = all_verbs[feature]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Stress Number'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Stress Number'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        #word_predict = model.predict(count_vect.transform(np.array(['ʌbɛmdinijɛst*ɛ*vijɛ'])))
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info


In [88]:
log_with_stress_feature = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [89]:
log_with_stress_feature

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999313045270317,
  'confusion matrix': array([[11116,     0],
         [    1,  3440]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.8595177577797624,
  'confusion matrix': array([[ 190,    0,   60],
         [   1, 7166,  726],
         [  51, 1207, 5156]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.8414508483890911,
  'confusion matrix': array([[3162,   15,    0,    0],
         [   1, 4989,  100,    0],
         [   0,  270, 1884,  987],
         [   0,    0,  935, 2214]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999313045270317,
  'confusion matrix': array([[ 4488,     0],
         [    1, 10068]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9996565226351584,
  'confusion matrix': array([[7230,    2],
         [   3, 7322]])},
 'Person': {'model': LogisticRegression(max_it

In [90]:
all_verbs['First Character'] = all_verbs['Generated Verb-SR'].apply(lambda word: 0 if word[0]in ['a', 'o', 'u', 'i','ø', 'ɛ', 'ʌ', 'e'] else 1) #before : goes input variable

In [91]:
all_verbs[all_verbs['First Character'] == 1]

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character
256,256,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,me-*ne*-andʌd͡ʒ-i-em(ɛ),menaˈndʌd͡ʒijemɛ,a,0.250000,1
257,257,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,me-*ne*-andʌd͡ʒ-i-i,menaˈndʌd͡ʒii,a,0.307692,1
258,258,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,me-*ne*-andʌd͡ʒ-i-e,menaˈndʌd͡ʒije,a,0.285714,1
259,259,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,me-*ne*-andʌd͡ʒ-i-iɛ,menaˈndʌd͡ʒiijɛ,a,0.266667,1
260,260,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,me-*ne*-andʌd͡ʒ-i-om(ɛ),menaˈndʌd͡ʒijomɛ,a,0.250000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41157,41157,past,subjunctive,pluperfect,non-causative,affirmative,3sm,passive,intrans,be-van-i-ast-*abe*,bevɛnijastaˈbe,a,0.785714,1
41158,41158,past,subjunctive,pluperfect,non-causative,affirmative,3sf,passive,intrans,be-van-i-ast-*abiɛ*,bevɛnijɛstɛˈbijɛ,ɛ,0.687500,1
41159,41159,past,subjunctive,pluperfect,non-causative,affirmative,1p,passive,intrans,be-van-i-ast-*abimon*,bevɛnijɛstɛˈbimon,ɛ,0.647059,1
41160,41160,past,subjunctive,pluperfect,non-causative,affirmative,2p,passive,intrans,be-van-i-ast-*abion*,bevɛnijɛstɛˈbijon,ɛ,0.647059,1


In [92]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:


        model = LogisticRegression(max_iter=100000) 
        X = all_verbs[['Generated Verb-SR', 'First Character']] 
        y = all_verbs[feature]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['First Character'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['First Character'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info


In [93]:
log_with_first_character = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [94]:
log_with_first_character

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999313045270317,
  'confusion matrix': array([[11116,     0],
         [    1,  3440]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.8586934121041423,
  'confusion matrix': array([[ 189,    0,   61],
         [   1, 7174,  718],
         [  51, 1226, 5137]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.8408325891323761,
  'confusion matrix': array([[3158,   18,    1,    0],
         [   1, 4989,  100,    0],
         [   0,  271, 1881,  989],
         [   0,    0,  937, 2212]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999313045270317,
  'confusion matrix': array([[ 4488,     0],
         [    1, 10068]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9994504362162534,
  'confusion matrix': array([[7229,    3],
         [   5, 7320]])},
 'Person': {'model': LogisticRegression(max_it

In [95]:
all_verbs['Last Character'] = all_verbs['Generated Verb-SR'].apply(lambda word: word[-1]) #before : goes input variable

In [96]:
le = LabelEncoder()

all_verbs['Last Character Number'] = le.fit_transform(all_verbs['Last Character'])

In [97]:
all_verbs['Last Character Number']

0        15
1         5
2         3
3        15
4        15
         ..
44107    15
44108    15
44109     8
44110     8
44111     8
Name: Last Character Number, Length: 44112, dtype: int64

In [98]:
le = LabelEncoder()
all_verbs_encoded = all_verbs
all_verbs_encoded = all_verbs_encoded.apply(lambda col: le.fit_transform(col))

In [99]:
all_verbs_encoded

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number
0,0,1,1,0,1,1,1,1,0,9662,67,1,40,0,15,15
1,1,1,1,0,1,1,3,1,0,9664,62,1,46,0,5,5
2,2,1,1,0,1,1,6,1,0,9661,66,1,43,0,3,3
3,3,1,1,0,1,1,5,1,0,9665,63,1,41,0,15,15
4,4,1,1,0,1,1,0,1,0,9666,69,1,40,0,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44107,44107,0,2,3,1,0,6,0,1,30631,30545,6,60,0,15,15
44108,44108,0,2,3,1,0,5,0,1,30631,30545,6,60,0,15,15
44109,44109,0,2,3,1,0,0,0,1,30629,30542,6,55,0,8,8
44110,44110,0,2,3,1,0,2,0,1,30630,30541,6,55,0,8,8


In [100]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:
        model = LogisticRegression(max_iter=100000) 

        X = all_verbs[['Generated Verb-SR', 'Last Character Number']] 
        y = all_verbs[feature] #predicted/target varuiable
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info


In [101]:
logistic_reg_with_last_character = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [102]:
logistic_reg_with_last_character

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999313045270317,
  'confusion matrix': array([[11116,     0],
         [    1,  3440]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.8582812392663324,
  'confusion matrix': array([[ 193,    0,   57],
         [   1, 7173,  719],
         [  55, 1231, 5128]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.8411760664972179,
  'confusion matrix': array([[3161,   16,    0,    0],
         [   1, 4990,   99,    0],
         [   0,  271, 1883,  987],
         [   0,    0,  938, 2211]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999313045270317,
  'confusion matrix': array([[ 4488,     0],
         [    1, 10068]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9996565226351584,
  'confusion matrix': array([[7229,    3],
         [   2, 7323]])},
 'Person': {'model': LogisticRegression(max_it

In [103]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:
        model = LogisticRegression(max_iter=100000)
        X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character']] 
        y = all_verbs[feature] #predicted/target varuiable
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info


In [104]:
log_reg_with_first_and_last_character = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [105]:
log_reg_with_first_and_last_character

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999313045270317,
  'confusion matrix': array([[11116,     0],
         [    1,  3440]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.858212543793364,
  'confusion matrix': array([[ 190,    0,   60],
         [   1, 7174,  718],
         [  53, 1232, 5129]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.8414508483890911,
  'confusion matrix': array([[3162,   15,    0,    0],
         [   1, 4990,   99,    0],
         [   0,  272, 1885,  984],
         [   0,    0,  937, 2212]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999313045270317,
  'confusion matrix': array([[ 4488,     0],
         [    1, 10068]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9994504362162534,
  'confusion matrix': array([[7229,    3],
         [   5, 7320]])},
 'Person': {'model': LogisticRegression(max_ite

In [106]:
# as a last feature, I want to see if the number of vowels is relevant to the predictions

In [107]:
all_verbs['Vowels Quantity'] = all_verbs['Generated Verb-SR'].apply(lambda word: len(re.findall(r'[aouiøɛʌe]', word))) #before : goes input variable

In [108]:
all_verbs['Vowels Quantity']

0        6
1        5
2        5
3        6
4        6
        ..
44107    5
44108    5
44109    5
44110    5
44111    5
Name: Vowels Quantity, Length: 44112, dtype: int64

In [109]:
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692,0,ɛ,15,6
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000,0,i,5,5
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636,0,e,3,5
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333,0,ɛ,15,6
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692,0,ɛ,15,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44107,44107,past,subjunctive,pluperfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
44108,44108,past,subjunctive,pluperfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
44109,44109,past,subjunctive,pluperfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ,0.500000,0,n,8,5
44110,44110,past,subjunctive,pluperfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ,0.500000,0,n,8,5


In [110]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:

        model = LogisticRegression(max_iter=100000) 
        X = all_verbs[['Generated Verb-SR', 'Vowels Quantity']] 
        y = all_verbs[feature] #predicted/target varuiable
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Vowels Quantity'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train) 
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info
  


In [111]:
log_reg_with_vowel_quantity = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [112]:
log_reg_with_vowel_quantity

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999313045270317,
  'confusion matrix': array([[11116,     0],
         [    1,  3440]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.8587621075771107,
  'confusion matrix': array([[ 193,    0,   57],
         [   1, 7172,  720],
         [  52, 1226, 5136]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.8407638936594078,
  'confusion matrix': array([[3158,   18,    1,    0],
         [   1, 4989,  100,    0],
         [   0,  271, 1882,  988],
         [   0,    0,  939, 2210]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999313045270317,
  'confusion matrix': array([[ 4488,     0],
         [    1, 10068]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9994504362162534,
  'confusion matrix': array([[7229,    3],
         [   5, 7320]])},
 'Person': {'model': LogisticRegression(max_it

In [113]:
## putting all features together in a LR model:

In [114]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character', 'Vowels Quantity']] 
    y = all_verbs[features] 
    X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X, y, test_size = 0.33, random_state = 1)
    count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
    X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
    X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1), X_test['Vowels Quantity'].values.reshape(-1,1)]) #no need to fit again, otherwise what it sees on training and test data may differ and end up with an erorr
    for feature in features:
        model = LogisticRegression(max_iter=100000) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train_pre_encoded[feature]) 
        y_test = le.transform(y_test_pre_encoded[feature])
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info
    # this code is imroved: most reliable for my classification models (it is faster as some parts have gone out of the scope of the function)


In [115]:
log_reg_with_three_added_features = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [116]:
log_reg_with_three_added_features

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999313045270317,
  'confusion matrix': array([[11116,     0],
         [    1,  3440]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.8584873256852373,
  'confusion matrix': array([[ 193,    0,   57],
         [   1, 7174,  718],
         [  51, 1233, 5130]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.8415195438620595,
  'confusion matrix': array([[3162,   15,    0,    0],
         [   1, 4989,  100,    0],
         [   0,  272, 1880,  989],
         [   0,    0,  930, 2219]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999313045270317,
  'confusion matrix': array([[ 4488,     0],
         [    1, 10068]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9994504362162534,
  'confusion matrix': array([[7230,    2],
         [   6, 7319]])},
 'Person': {'model': LogisticRegression(max_it

In [117]:
all_verbs.head()

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692,0,ɛ,15,6
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000,0,i,5,5
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636,0,e,3,5
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333,0,ɛ,15,6
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692,0,ɛ,15,6


In [118]:
tense_model = log_reg_with_three_added_features['Tense']['model'] #separating the models from each other

In [119]:
X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character', 'Vowels Quantity']] #select the list by two square brackets
y = all_verbs['Tense'] 
words_train, words_test, X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X,X, y, test_size = 0.33, random_state = 1) # splitting X twice , in one of them keep the original in one vectorized. 
count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1), X_test['Vowels Quantity'].values.reshape(-1,1)]) #no need to fit again, otherwise what it sees on training and test data may differ and end up with an erorr
le = LabelEncoder()
y_train = le.fit_transform(y_train_pre_encoded) 
y_test = le.transform(y_test_pre_encoded)
print(le.classes_)

['past' 'present']


In [120]:
# I only pulled out tense, and now we are predict on the test set top see where the misclassifications occured
tense_model.predict(X_test) # compare these with y_test

array([1, 1, 1, ..., 0, 0, 0])

In [121]:
y_test

array([1, 1, 1, ..., 0, 0, 0])

In [122]:
y_test !=  tense_model.predict(X_test) # pairwise comparison

array([False, False, False, ..., False, False, False])

In [123]:
words_test[y_test !=  tense_model.predict(X_test)] # these will keep only the rows that do not agree

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity
29785,gaˈrdemɛ,15,1,3



aspect_model = log_reg_with_three_added_features['Aspect']['model']
X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character', 'Vowels Quantity']] 
words_train, words_test, X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X,X, y, test_size = 0.33, random_state = 1) 
count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1), X_test['Vowels Quantity'].values.reshape(-1,1)]) #no need to fit again, otherwise what it sees on training and test data may differ and end up with an erorr
le = LabelEncoder()
y_train = le.fit_transform(y_train_pre_encoded)
y_test = le.transform(y_test_pre_encoded)
print(le.classes_)
aspect_model.predict(X_test)
aspect_mismatches = words_test[y_test !=  aspect_model.predict(X_test)]
aspect_mismatches['y_train_pre_encoded'] = y_train_pre_encoded
aspect_mismatches.head()

In [124]:
type(words_test)

pandas.core.frame.DataFrame

In [125]:
type(X_test)

scipy.sparse.coo.coo_matrix

In [126]:
aspect_model.predict(X_test).size

14557

In [127]:
le.classes_

array(['past', 'present'], dtype=object)

words_test['predicted'] = modaliy_model.predict(X_test) #made a new column

words_test['predicted_aspect'] = words_test['predicted'].apply(lambda x: le.classes_[x]) #index lookup and doing the opposite of vectorization. Give me the class at index 0 or 1 ...

words_test['actual_aspect'] = all_verbs.iloc[words_test.index]['Aspect'] #word test preserves the indexes from the original data frame (see on the left): the indexe are not in a paarticular order. so you can use this index to go back to 

original df and pass that indexes to df and look up what the values are for the rows with that index, we only care about values of aspect columns, we wanna get the aspect value


In [131]:
def LG_model():
        dataframes = {}
        features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
        for feature in features:
                feature_model = log_reg_with_three_added_features[feature]['model']
                X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character', 'Vowels Quantity']] 
                y = all_verbs[feature]
                words_train, words_test, X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X,X, y, test_size = 0.33, random_state = 1) 
                count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
                X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
                    
                X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1), X_test['Vowels Quantity'].values.reshape(-1,1)]) #no need to fit again, otherwise what it sees on training and test data may differ and end up with an erorr
                le = LabelEncoder()
                y_train = le.fit_transform(y_train_pre_encoded)
                y_test = le.transform(y_test_pre_encoded)
                print(le.classes_)
                words_test['predicted'] = feature_model.predict(X_test) 
                words_test['predicted ' + feature] = words_test['predicted'].apply(lambda x: le.classes_[x])
                words_test['actual ' + feature] = all_verbs.iloc[words_test.index][feature]
                dataframes[feature] = words_test
        return dataframes


In [132]:
dfs = LG_model()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [133]:
dfs['Tense']

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity,predicted,predicted Tense,actual Tense
35542,ʌnjɛrijʌˈ,19,0,4,1,present,present
24269,neˈvʌzdinii,5,1,5,1,present,present
40401,ʌmneˈt͡ʃʌltinijɛ,15,0,6,1,present,present
25625,bʌvid͡ʒijɛˈstimon,8,1,6,0,past,past
40778,ʌtʌʃɛstɛˈbi,5,0,5,0,past,past
...,...,...,...,...,...,...,...
24902,mʌˈr,9,1,1,1,present,present
40009,ʌʃɢʌldɛˈbimɛ,15,0,5,0,past,past
42542,ʌχosdinijɛˈstijon,8,0,7,0,past,past
30875,op:ardenɛstɛˈviʃon,8,0,7,0,past,past


In [134]:
base_df = dfs['Tense'].copy()
base_df['predicted Modality'] = dfs['Modality']['predicted Modality']
base_df['actual Modality'] = dfs['Modality']['actual Modality']
base_df['predicted Aspect'] = dfs['Aspect']['predicted Aspect']
base_df['actual Aspect'] = dfs['Aspect']['actual Aspect']
base_df['predicted Causation'] = dfs['Causation']['predicted Causation']
base_df['actual Causation'] = dfs['Causation']['actual Causation']
base_df['predicted Polarity'] = dfs['Polarity']['predicted Polarity']
base_df['actual Polarity'] = dfs['Polarity']['actual Polarity']
base_df['predicted Person'] = dfs['Person']['predicted Person']
base_df['actual Person'] = dfs['Person']['actual Person']
base_df['predicted Transitivity'] = dfs['Transitivity']['predicted Transitivity']
base_df['actual Transitivity'] = dfs['Transitivity']['actual Transitivity']

In [135]:
base_df.shape

(14557, 19)

In [136]:
base_df

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity,predicted,predicted Tense,actual Tense,predicted Modality,actual Modality,predicted Aspect,actual Aspect,predicted Causation,actual Causation,predicted Polarity,actual Polarity,predicted Person,actual Person,predicted Transitivity,actual Transitivity
35542,ʌnjɛrijʌˈ,19,0,4,1,present,present,indicative,indicative,imperfective,imperfective,non-causative,non-causative,affirmative,affirmative,2p,2p,intrans,intrans
24269,neˈvʌzdinii,5,1,5,1,present,present,subjunctive,subjunctive,neutral,neutral,causative,causative,negative,negative,2s,2s,intrans,intrans
40401,ʌmneˈt͡ʃʌltinijɛ,15,0,6,1,present,present,indicative,indicative,imperfective,imperfective,causative,causative,negative,negative,3sf,3sf,trans,trans
25625,bʌvid͡ʒijɛˈstimon,8,1,6,0,past,past,indicative,subjunctive,neutral,neutral,non-causative,non-causative,affirmative,affirmative,1p,1p,intrans,intrans
40778,ʌtʌʃɛstɛˈbi,5,0,5,0,past,past,subjunctive,subjunctive,pluperfect,pluperfect,non-causative,non-causative,affirmative,affirmative,2s,2s,trans,trans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24902,mʌˈr,9,1,1,1,present,present,imperative,imperative,neutral,neutral,non-causative,non-causative,affirmative,negative,2s,2s,trans,trans
40009,ʌʃɢʌldɛˈbimɛ,15,0,5,0,past,past,subjunctive,subjunctive,perfect,pluperfect,non-causative,non-causative,affirmative,affirmative,1s,1s,trans,trans
42542,ʌχosdinijɛˈstijon,8,0,7,0,past,past,indicative,subjunctive,neutral,neutral,causative,causative,affirmative,affirmative,2p,2p,intrans,intrans
30875,op:ardenɛstɛˈviʃon,8,0,7,0,past,past,indicative,indicative,pluperfect,pluperfect,causative,causative,affirmative,affirmative,3p,3p,trans,trans


In [138]:
base_df.to_csv('./base_df.csv', encoding = 'utf-8-sig', index = False)

In [139]:
base_df

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity,predicted,predicted Tense,actual Tense,predicted Modality,actual Modality,predicted Aspect,actual Aspect,predicted Causation,actual Causation,predicted Polarity,actual Polarity,predicted Person,actual Person,predicted Transitivity,actual Transitivity
35542,ʌnjɛrijʌˈ,19,0,4,1,present,present,indicative,indicative,imperfective,imperfective,non-causative,non-causative,affirmative,affirmative,2p,2p,intrans,intrans
24269,neˈvʌzdinii,5,1,5,1,present,present,subjunctive,subjunctive,neutral,neutral,causative,causative,negative,negative,2s,2s,intrans,intrans
40401,ʌmneˈt͡ʃʌltinijɛ,15,0,6,1,present,present,indicative,indicative,imperfective,imperfective,causative,causative,negative,negative,3sf,3sf,trans,trans
25625,bʌvid͡ʒijɛˈstimon,8,1,6,0,past,past,indicative,subjunctive,neutral,neutral,non-causative,non-causative,affirmative,affirmative,1p,1p,intrans,intrans
40778,ʌtʌʃɛstɛˈbi,5,0,5,0,past,past,subjunctive,subjunctive,pluperfect,pluperfect,non-causative,non-causative,affirmative,affirmative,2s,2s,trans,trans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24902,mʌˈr,9,1,1,1,present,present,imperative,imperative,neutral,neutral,non-causative,non-causative,affirmative,negative,2s,2s,trans,trans
40009,ʌʃɢʌldɛˈbimɛ,15,0,5,0,past,past,subjunctive,subjunctive,perfect,pluperfect,non-causative,non-causative,affirmative,affirmative,1s,1s,trans,trans
42542,ʌχosdinijɛˈstijon,8,0,7,0,past,past,indicative,subjunctive,neutral,neutral,causative,causative,affirmative,affirmative,2p,2p,intrans,intrans
30875,op:ardenɛstɛˈviʃon,8,0,7,0,past,past,indicative,indicative,pluperfect,pluperfect,causative,causative,affirmative,affirmative,3p,3p,trans,trans
